# 示例：缓存及加载功能
- 可以缓存及加载任何对象（数据、模型……）

In [1]:
from fastphm.data.FeatureExtractor import FeatureExtractor
from fastphm.data.labeler.RulLabeler import RulLabeler
from fastphm.data.loader.bearing.XJTULoader import XJTULoader
from fastphm.data.processor.RMSProcessor import RMSProcessor
from fastphm.data.stage.BearingStageCalculator import BearingStageCalculator
from fastphm.data.stage.fpt.ThreeSigmaFPTCalculator import ThreeSigmaFPTCalculator
from fastphm.util.Cache import Cache
from fastphm.model.pytorch.PytorchModel import PytorchModel
from fastphm.model.pytorch.basic.MLP import MLP

### 生成数据

In [2]:
data_loader = XJTULoader('D:\\data\\dataset\\XJTU-SY_Bearing_Datasets')
feature_extractor = FeatureExtractor(RMSProcessor(data_loader.continuum))
fpt_calculator = ThreeSigmaFPTCalculator()
stage_calculator = BearingStageCalculator(data_loader.continuum, fpt_calculator)

# 获取原始数据、特征数据、阶段数据
bearing = data_loader("Bearing1_3", 'Horizontal Vibration')
feature_extractor(bearing)
stage_calculator(bearing)

generator = RulLabeler(2048, time_ratio=60, is_from_fpt=False, is_rectified=True)
train_set = generator(bearing)

DEBUG - 17:43:54 >> 
<< Root directory of dataset: D:\data\dataset\XJTU-SY_Bearing_Datasets
	Bearing1_1, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_1
	Bearing1_2, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_2
	Bearing1_3, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_3
	Bearing1_4, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_4
	Bearing1_5, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_5
	Bearing2_1, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_1
	Bearing2_2, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_2
	Bearing2_3, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_3
	Bearing2_4, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_4
	Bearing2_5, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_5
	Bearing3_1, location: D:\data\dataset\XJTU-SY_Bearing_Dat

### 缓存数据

In [3]:
Cache.save(train_set, 'train_set')

DEBUG - 17:43:57 >> Start generating cache file: .\cache\train_set.pkl
DEBUG - 17:43:57 >> Successfully generated cache file: .\cache\train_set.pkl


### 加载数据

In [4]:
train_set = Cache.load('train_set')

DEBUG - 17:43:57 >> Start loading cache file: .\cache\train_set.pkl
DEBUG - 17:43:57 >> Successfully loaded cache file: .\cache\train_set.pkl


In [5]:
train_set.x

array([[ -0.77383518,   0.1305342 ,   0.1555443 , ...,   0.1136541 ,
         -0.54583549,   0.42616129],
       [ -0.38670301,  -0.3725886 ,  -0.74648857, ...,  -0.0345588 ,
         -0.70579052,  -0.79065561],
       [ -0.27912861,   0.52905079,  -0.2827883 , ...,  -0.63660137,
          0.25235411,   0.45748949],
       ...,
       [ -0.48875809,   0.66200487,  -2.59971619, ..., -11.16659641,
         -3.66904736,  -6.93216324],
       [ -2.04831362,  -0.154233  ,   3.88391018, ...,  -9.79976654,
          1.93562508,  -7.22073317],
       [ -3.97603512,   2.60374546,  -0.52655939, ...,  -5.99588156,
         -4.60753441, -10.23435593]])

### 生成模型并训练

In [6]:
model = PytorchModel(MLP(2048,16,1))
model.train(train_set, epochs=10, batch_size=256, lr=0.01, weight_decay=0.01)

INFO - 17:43:57 >> 
<< Successfully initialized model:
	class: MLP
	device: cuda
	dtype: torch.float32
INFO - 17:43:58 >> 
<< Start training model:
	loss function: MSELoss
	optimizer: Adam
	learning rate: 0.01
	weight decay: 0.01
	batch size: 256
DEBUG - 17:43:58 >> Epoch 1/10, Loss: 2.9730943799
DEBUG - 17:43:58 >> Epoch 2/10, Loss: 2.6427245855
DEBUG - 17:43:58 >> Epoch 3/10, Loss: 0.9938627124
DEBUG - 17:43:58 >> Epoch 4/10, Loss: 0.4982388645
DEBUG - 17:43:58 >> Epoch 5/10, Loss: 0.3173443541
DEBUG - 17:43:58 >> Epoch 6/10, Loss: 0.2191797912
DEBUG - 17:43:58 >> Epoch 7/10, Loss: 0.1704204008
DEBUG - 17:43:58 >> Epoch 8/10, Loss: 0.1305305324
DEBUG - 17:43:58 >> Epoch 9/10, Loss: 0.1046425104
DEBUG - 17:43:58 >> Epoch 10/10, Loss: 0.0864575386
INFO - 17:43:58 >> Model training completed!!!


### 缓存模型

In [7]:
Cache.save(model, 'model')

DEBUG - 17:43:58 >> Start generating cache file: .\cache\model.pkl
DEBUG - 17:43:58 >> Successfully generated cache file: .\cache\model.pkl


### 加载模型

In [8]:
model = Cache.load('model')

DEBUG - 17:43:58 >> Start loading cache file: .\cache\model.pkl
DEBUG - 17:43:58 >> Successfully loaded cache file: .\cache\model.pkl
